# Import package 


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ot 
import numba as nb

# Sinkhorn solver and Lp solver for GOPT problem 
See Equation (64)(65)(66)(67) for the derivation of this Algorithm. 
See Equation Remark 7 and (75) for the special case. 

In [11]:

@nb.njit(fastmath=True,cache=True)
def cost_matrix_d(X,Y,p=2):
    '''
    input: 
        X: (n,) float np array
        Y: (m,) float np array
    output:
        M: n*m matrix, M_ij=c(X_i,Y_j) where c is defined by cost_function.
    
    '''
    n,d=X.shape
    m,d=Y.shape
    M=np.empty((n,m))
    for i in nb.prange(n):
        M[i]=np.sum((X[i]-Y)**p,1)
    return M

def opt_cost(gamma,C,Lambda1,Lambda2,mu,nu):
    trans_cost=np.sum(C*gamma)
    
    penalty_d=np.sum(Lambda1*(np.abs(mu-gamma.sum(1))))
    penalty_c=np.sum(Lambda2*(np.abs(nu-gamma.sum(0))))
    return trans_cost,penalty_d+penalty_c

def sinkhorn_knopp_gopt(mu, nu, M, Lambda1,Lambda2, reg=0.1, numItermax=1000,F1='PTV',F2='PTV'):
    r"""
    Solve the entropic regularization optimal transport problem and return the OT matrix

    The function solves the following optimization problem:

    .. math::
        \gamma = \mathop{\arg \min}_\gamma \quad \langle \gamma, \mathbf{M} \rangle_F +
        \mathrm{reg}\cdot\Omega(\gamma)

        s.t. \ \gamma \mathbf{1} &= \mathbf{a}

             \gamma^T \mathbf{1} &= \mathbf{b}

             \gamma &\geq 0
    where :

    - :math:`\mathbf{M}` is the (`dia_mu`, `dia_nu`) metric cost matrix
    - :math:`\Omega` is the entropic regularization term
      :math:`\Omega(\gamma)=\sum_{i,j} \gamma_{i,j}\log(\gamma_{i,j})`
    - :math:`\mathbf{a}` and :math:`\mathbf{b}` are source and target
      weights (histograms, both sum to 1)

    The algorithm used for solving the problem is the Sinkhorn-Knopp
    matrix scaling algorithm as proposed in :ref:`[2] <references-sinkhorn-knopp>`


    Parameters
    ----------
    mu : array-like, shape (mu,) float64
        samples weights in the source domain
    nu : array-like, shape (nu,) float64
    M : array-like, shape (dia_mu, dia_nu) float64
        loss matrix
    reg : float
        Regularization term >0
    numItermax : int64, optional
        Max number of iterations
    stopThr : float64, optional
        Stop threshold on error (>0)
    lambda1: (n,) array
    lambda2: (m,) array
    Returns
    -------
    gamma : array-like, shape (dim_mu, dim_nu)
        Optimal transportation matrix for the given parameters

    Examples
    --------

    >>> import ot
    >>> mu=[.5, .5]
    >>> nu=[.5, .5]
    >>> M=[[0., 1.], [1., 0.]]
    >>> ot.sinkhorn(a, b, M, 1)
    array([[0.36552929, 0.13447071],
           [0.13447071, 0.36552929]])


    .. _references-sinkhorn-knopp:
    References
    ----------

    .. [2] M. Cuturi, Sinkhorn Distances : Lightspeed Computation
        of Optimal Transport, Advances in Neural Information
        Processing Systems (NIPS) 26, 2013


    See Also
    --------
    ot.lp.emd : Unregularized OT
    ot.optim.cg : General regularized OT

    """
    

    # init data
    dim_mu = mu.shape[0]
    dim_nu = nu.shape[0]
    stopThr=1e-9
    
    #initialize u,v 
    u = np.ones(dim_mu, dtype=float)
    v = np.ones(dim_nu, dtype=float)
    
    if Lambda1.shape[0]==1:
        Lambda1=np.full(n,Lambda1[0])
    if Lambda2.shape[0]==1:
        Lambda2=np.full(m,Lambda2[0])
    if Lambda1.shape[0] != n:
        raise ValueError("Lambda1 should have the same number of rows as n.")
    if Lambda2.shape[0] != m:
        raise ValueError("Lambda2 should have the same number of rows as m.")
#        warnings.warn("Lambda2 should have the same number of rows as m.", UserWarning)

    K = np.exp(-M/reg)
    Lambda1_exp_u,Lambda1_exp_l=np.exp(Lambda1/reg),np.exp(-Lambda1/reg)
    Lambda2_exp_u,Lambda2_exp_l=np.exp(Lambda2/reg),np.exp(-Lambda2/reg)
    # initialize proxdiv_F1, proxdiv_F2    
    if F1=='TV':
        def proxdiv_F1(s): 
            return np.clip(mu/s,Lambda1_exp_l,Lambda1_exp_u)
    elif F1=='PTV':
        def proxdiv_F1(s):
            return np.minimum(mu/s,Lambda1_exp_u)
        
    elif F1=='TV0':
        def proxdiv_F1(s):
            return np.ones(s.shape[0], dtype=float)
        
    elif F1=='inf':
        def proxdiv_F1(s):             
            return mu/s

    if F2=='TV':
        def proxdiv_F2(s):
            return np.clip(nu/s,Lambda2_exp_l,Lambda2_exp_u)
    elif F2=='PTV':
        def proxdiv_F2(s):
            return np.minimum(nu/s,Lambda2_exp_u)

    elif F2=='TV0':
        def proxdiv_F2(s):             
            return np.ones(s.shape[0], dtype=float)
    elif F2=='inf':
        def proxdiv_F2(s):             
            return nu/s


    if F1 not in ['TV','PTV']:
        raise ValueError("F1 should be either TV or PTV.")
    if F2 not in ['TV','PTV']:
        raise ValueError("F2 should be either TV or PTV.")
        


    
    
    for ii in range(numItermax):
        u_pre=u.copy()
        v_pre=v.copy()
        v = proxdiv_F2(np.dot(K.T,u))
        u = proxdiv_F1(np.dot(K,v))
        if ii % 10 == 0:
            # we can speed up the process by checking for the error only all
            # the 10th iterations
            err = np.linalg.norm(u_pre - u)+np.linalg.norm(v_pre - v)  # violation of marginal
            if err < stopThr:
                break
    gamma=np.expand_dims(u,1)*(K*v.T)

    return gamma

@nb.njit(cache=True)
def sinkhorn_knopp_gopt_numba(mu, nu, M, Lambda1,Lambda2, reg=0.1, numItermax=1000,F1='TV',F2='TV'):
    r"""
    Solve the entropic regularization optimal transport problem and return the OT matrix

    The function solves the following optimization problem:

    .. math::
        \gamma = \mathop{\arg \min}_\gamma \quad \langle \gamma, \mathbf{M} \rangle_F +
        \mathrm{reg}\cdot\Omega(\gamma)

        s.t. \ \gamma \mathbf{1} &= \mathbf{a}

             \gamma^T \mathbf{1} &= \mathbf{b}

             \gamma &\geq 0
    where :

    - :math:`\mathbf{M}` is the (`dia_mu`, `dia_nu`) metric cost matrix
    - :math:`\Omega` is the entropic regularization term
      :math:`\Omega(\gamma)=\sum_{i,j} \gamma_{i,j}\log(\gamma_{i,j})`
    - :math:`\mathbf{a}` and :math:`\mathbf{b}` are source and target
      weights (histograms, both sum to 1)

    The algorithm used for solving the problem is the Sinkhorn-Knopp
    matrix scaling algorithm as proposed in :ref:`[2] <references-sinkhorn-knopp>`


    Parameters
    ----------
    mu : array-like, shape (mu,) float64
        samples weights in the source domain
    nu : array-like, shape (nu,) float64
    M : array-like, shape (dia_mu, dia_nu) float64
        loss matrix
    reg : float
        Regularization term >0
    numItermax : int64, optional
        Max number of iterations
    stopThr : float64, optional
        Stop threshold on error (>0)
    lambda1: (n,) array
    lambda2: (m,) array
    Returns
    -------
    gamma : array-like, shape (dim_mu, dim_nu)
        Optimal transportation matrix for the given parameters

    Examples
    --------

    >>> import ot
    >>> mu=[.5, .5]
    >>> nu=[.5, .5]
    >>> M=[[0., 1.], [1., 0.]]
    >>> ot.sinkhorn(a, b, M, 1)
    array([[0.36552929, 0.13447071],
           [0.13447071, 0.36552929]])


    .. _references-sinkhorn-knopp:
    References
    ----------

    .. [2] M. Cuturi, Sinkhorn Distances : Lightspeed Computation
        of Optimal Transport, Advances in Neural Information
        Processing Systems (NIPS) 26, 2013


    See Also
    --------
    ot.lp.emd : Unregularized OT
    ot.optim.cg : General regularized OT

    """
    


    dim_mu = mu.shape[0]
    dim_nu = nu.shape[0]
    stopThr=1e-10
    
    #initialize u,v 
    u = np.ones(dim_mu, dtype=float)
    v = np.ones(dim_nu, dtype=float)
    
    
    #check lambda1 lambda2
    if Lambda1.shape[0]==1:
        Lambda1=np.full(n,Lambda1[0])
    if Lambda2.shape[0]==1:
        Lambda2=np.full(m,Lambda2[0])
    if Lambda1.shape[0] != n:
        raise ValueError("Lambda1 should have the same number of rows as n.")
    if Lambda2.shape[0] != m:
        raise ValueError("Lambda2 should have the same number of rows as m.")

    
    # initialize K 
    K = np.exp(-M/reg)    
    Lambda1_exp_u,Lambda1_exp_l=np.exp(Lambda1/reg),np.exp(-Lambda1/reg)
    Lambda2_exp_u,Lambda2_exp_l=np.exp(Lambda2/reg),np.exp(-Lambda2/reg)
    
    

    
    # initilize proximal-divider operator 
    if F1=='TV':
        def proxdiv_F1(s,mu,Lambda1_exp_l,Lambda1_exp_u):             
            return np.clip(mu/s,Lambda1_exp_l,Lambda1_exp_u)
    elif F1=='PTV':
        def proxdiv_F1(s,mu,Lambda1_exp_l,Lambda1_exp_u):
            return np.minimum(mu/s,Lambda1_exp_u)
    elif F1=='TV0':
        def proxdiv_F1(s,mu,Lambda1_exp_l,Lambda1_exp_u):
            return np.ones(s.shape[0], dtype=float)
    elif F1=='inf':
        def proxdiv_F1(s,mu,Lambda1_exp_l,Lambda1_exp_u):             
            return mu/s
        

    if F2=='TV':
        def proxdiv_F2(s,nu,Lambda2_exp_l,Lambda2_exp_u):
            return np.clip(nu/s,Lambda2_exp_l,Lambda2_exp_u)
    elif F2=='PTV':
        def proxdiv_F2(s,nu,Lambda2_exp_l,Lambda2_exp_u):
           return np.minimum(nu/s,Lambda2_exp_u)
    elif F2=='TV0':
        def proxdiv_F2(s,nu,Lambda2_exp_l,Lambda2_exp_u):             
            return np.ones(s.shape[0], dtype=float)
    elif F2=='inf':
        def proxdiv_F2(s,nu,Lambda1_exp_l,Lambda1_exp_u):             
            return nu/s
        
    if F1 not in ['TV','PTV','TV0','inf']:
        raise ValueError("F1 should be either TV or PTV.")
    if F2 not in ['TV','PTV','TV0','inf']:
        raise ValueError("F2 should be either TV or PTV.")
                           
    # main loop
    for ii in range(numItermax):
        u_pre=u.copy()
        v_pre=v.copy()
        v = proxdiv_F2(np.dot(K.T,u),nu,Lambda2_exp_l,Lambda2_exp_u)
        u = proxdiv_F1(np.dot(K,v)  ,mu,Lambda1_exp_l,Lambda1_exp_u)
        if ii % 10 == 0:
            err = np.linalg.norm(u_pre - u)+np.linalg.norm(v_pre - v)  # violation of marginal
            if err < stopThr:
                break
    gamma=np.expand_dims(u,1)*(K*v.T)
    
    return gamma


def gopt_lp(mu,nu,M,Lambda1,Lambda2,numItermax=100000,numThreads=1):
    """
    Solves the partial optimal transport problem
    and returns the OT plan by linear programming in PythonOT 
    
    Parameters
    ----------
    mu : np.ndarray (dim_mu,) float64 
        Unnormalized histogram of dimension `dia_mu`
    nu : np.ndarray (dim_nu,) float64
        Unnormalized histograms of dimension `dia_nu`
    M : np.ndarray (dim_mu, dim_nu) float64
        cost matrix
    reg : float
        Regularization term > 0
    numItermax : int64, optional
        Max number of iterations


    Returns
    -------
    gamma : (dim_mu, dim_nu) ndarray
        Optimal transportation matrix for the given parameters
    cost : float64
        
    """
    n,m=M.shape 
     # check Lambda1,Lambda2
    Lambda1,Lambda2=np.array([Lambda1]),np.array([Lambda2])
    if Lambda1.shape[0]==1:
        Lambda1=np.full(n,Lambda1[0])
    if Lambda2.shape[0]==1:
        Lambda2=np.full(m,Lambda2[0])
    if Lambda1.shape[0] != n:
        raise ValueError("Lambda1 should have the same number of rows as n.")
    if Lambda2.shape[0] != m:
        raise ValueError("Lambda2 should have the same number of rows as m.")
#        warnings.warn("Lambda2 should have the same number of rows as m.", UserWarning)


    mu1=np.zeros(n+1)
    nu1=np.zeros(m+1)
    mu1[0:n]=mu
    nu1[0:m]=nu
    mu1[-1]=np.sum(nu)
    nu1[-1]=np.sum(mu)
    M1=np.zeros((n+1,m+1),dtype=np.float64)
    Lambda_matrix=np.expand_dims(Lambda1,1)+Lambda2
    M1[0:n,0:m]=M-Lambda_matrix
    gamma1=ot.lp.emd(mu1,nu1,M1,numItermax=numItermax,numThreads=numThreads)
    gamma=gamma1[0:n,0:m]
   
    return gamma








In [13]:
#  test the gopt solvers in a toy example: 

n,m=10,15
X=np.random.rand(n,2)*2
Y=np.random.rand(m,2)*2
C=cost_matrix_d(X,Y)
mu=np.random.rand(n)*4
mu/=mu.sum()
nu=np.random.rand(m)*4

nu/=nu.sum()*1.2
Lambda1=np.abs(np.random.rand(n))/2+.1
Lambda2=np.abs(np.random.rand(m))/2+.2 


gamma_sinkhorn=sinkhorn_knopp_gopt_numba(mu, nu, C, Lambda1,Lambda2, reg=0.01, numItermax=4000,F1='TV',F2='TV')
tranas_cost_sinkhorn,penalty_sinkhorn=opt_cost(gamma_sinkhorn,C,Lambda1,Lambda2,mu,nu)
gamma_lp=gopt_lp(mu, nu, C, Lambda1,Lambda2, numItermax=2000)
tranas_cost_lp,penalty_lp=opt_cost(gamma_lp,C,Lambda1,Lambda2,mu,nu)
print('opt_trans_cost and mass_penalty from Sinkhorn is %0.2f and %0.2f'%(tranas_cost_sinkhorn,penalty_sinkhorn))
# print('mass of |Gamma1-mu| is',np.sum(np.abs(gamma_sinkhorn.sum(1)-mu)))
# print('mass of |Gamma2-nu| is',np.sum(np.abs(gamma_sinkhorn.sum(0)-nu)))
print('opt_trans_cost and mass_penalty from lp with mass constraint is  %0.2f and %0.2f'% (tranas_cost_lp,penalty_lp))

# print('mass of |Gamma1-mu| is',np.sum(np.abs(gamma_lp.sum(1)-mu)))
# print('mass of |Gamma2-nu| is',np.sum(np.abs(gamma_lp.sum(0)-nu)))

opt_trans_cost and mass_penalty from Sinkhorn is 0.12 and 0.23
opt_trans_cost and mass_penalty from lp with mass constraint is  0.12 and 0.22


# Sinkhorn and Lp solvers for mass-constraint OPT problem

In [57]:
 
@nb.njit(cache=True)
def sinkhorn_knopp_mopt(mu, nu, M, mass, reg=0.1, numItermax=100000):
    r"""
    (we modify the code in PythonOT) 
    Solves the partial optimal transport problem
    and returns the OT plan vis Sinkhorn algorithm (we modify the code in PythonOT)

    The function considers the following problem:

    .. math::
        \gamma = \mathop{\arg \min}_\gamma \quad \langle \gamma,
                 \mathbf{M} \rangle_F + \mathrm{reg} \cdot\Omega(\gamma)

        s.t. \gamma \mathbf{1} &\leq \mathbf{a} \\
             \gamma^T \mathbf{1} &\leq \mathbf{b} \\
             \gamma &\geq 0 \\
             \mathbf{1}^T \gamma^T \mathbf{1} = m
             &\leq \min\{\|\mathbf{a}\|_1, \|\mathbf{b}\|_1\} \\

    where :

    - :math:`\mathbf{M}` is the metric cost matrix
    - :math:`\Omega`  is the entropic regularization term,
      :math:`\Omega=\sum_{i,j} \gamma_{i,j}\log(\gamma_{i,j})`
    - :math:`\mathbf{a}` and :math:`\mathbf{b}` are the sample weights
    - `m` is the amount of mass to be transported

    The formulation of the problem has been proposed in
    :ref:`[3] <references-entropic-partial-wasserstein>` (prop. 5)


    Parameters
    ----------
    mu : np.ndarray (dia_mu,) float64
        Unnormalized histogram of dimension `dia_mu`
    b : np.ndarray (dia_nu,) float64
        Unnormalized histograms of dimension `dia_nu`
    M : np.ndarray (dia_mu, dia_nu)
        cost matrix
    reg : float
        Regularization term > 0
    mass : float64, optional
        Amount of mass to be transported
    numItermax : int64, optional
        Max number of iterations
    stopThr : float64, optional
        Stop threshold on error (>0)
    verbose : bool, optional
        Print information along iterations
    log : bool, optional
        record log if True


    Returns
    -------
    gamma : (dia_mu, dia_nu) ndarray
        Optimal transportation matrix for the given parameters
    log : dict
        log dictionary returned only if `log` is `True`


    Examples
    --------
    >>> import ot
    >>> mu = [.1, .2]
    >>> nu = [.1, .1]
    >>> M = [[0., 1.], [2., 3.]]
    >>> np.round(entropic_partial_wasserstein(a, b, M, 1, 0.1), 2)
    array([[0.06, 0.02],
           [0.01, 0.  ]])


    .. _references-entropic-partial-wasserstein:
    References
    ----------
    .. [3] Benamou, J. D., Carlier, G., Cuturi, M., Nenna, L., & Peyré, G.
       (2015). Iterative Bregman projections for regularized transportation
       problems. SIAM Journal on Scientific Computing, 37(2), A1111-A1138.

    See Also
    --------
    ot.partial.partial_wasserstein: exact Partial Wasserstein
    """


    dim_mu, dim_nu = M.shape
    dx = np.ones(dim_mu, dtype=np.float64)
    dy = np.ones(dim_nu, dtype=np.float64)
    stopThr=1e-13
            

    # Next 3 lines equivalent to K=np.exp(-M/reg), but faster to compute
    K=np.exp(-M/reg)    
    K=K*mass / np.sum(K) # make the total mass of K to be mass
    
    err, cpt = 1, 0
    q1 = np.ones(K.shape)
    q2 = np.ones(K.shape)
    q3 = np.ones(K.shape)
    

    if mass < 0:
        raise ValueError("Problem infeasible. Parameter mass should be greater"
                         " than 0.")
    elif mass > min(np.sum(mu), np.sum(nu)):
        raise ValueError("Problem infeasible. Parameter m should lower or"
                         " equal than min(|mu|_1, |nu|_1).")
        

    while (err > stopThr and cpt < numItermax):
        Kprev = K
        K = K * q1
        K1 = np.dot(np.diag(np.minimum(mu / np.sum(K, axis=1), dx)), K)
        q1 = q1 * Kprev / K1
        K1prev = K1
        K1 = K1 * q2
        K2 = np.dot(K1, np.diag(np.minimum(nu / np.sum(K1, axis=0), dy)))
        q2 = q2 * K1prev / K2
        K2prev = K2
        K2 = K2 * q3
        K = K2 * (mass / np.sum(K2))
        q3 = q3 * K2prev / K


        if cpt % 10 == 0:
            err = np.linalg.norm(Kprev - K)

        cpt = cpt + 1
    if cpt==numItermax-1:
        print('warning, maximum iteration reached')
    return K


def mopt_lp(mu, nu, M, mass, numItermax=100000,numThreads=1):
    r"""
    (we modify the code in PythonOT) 
    Solves the partial optimal transport problem
    and returns the OT plan vis Sinkhorn algorithm (we modify the code in PythonOT)

    The function considers the following problem:

    .. math::
        \gamma = \mathop{\arg \min}_\gamma \quad \langle \gamma,
                 \mathbf{M} \rangle_F + \mathrm{reg} \cdot\Omega(\gamma)

        s.t. \gamma \mathbf{1} &\leq \mathbf{a} \\
             \gamma^T \mathbf{1} &\leq \mathbf{b} \\
             \gamma &\geq 0 \\
             \mathbf{1}^T \gamma^T \mathbf{1} = m
             &\leq \min\{\|\mathbf{a}\|_1, \|\mathbf{b}\|_1\} \\

    where :

    - :math:`\mathbf{M}` is the metric cost matrix
    - :math:`\Omega`  is the entropic regularization term,
      :math:`\Omega=\sum_{i,j} \gamma_{i,j}\log(\gamma_{i,j})`
    - :math:`\mathbf{a}` and :math:`\mathbf{b}` are the sample weights
    - `m` is the amount of mass to be transported

    The formulation of the problem has been proposed in
    :ref:`[3] <references-entropic-partial-wasserstein>` (prop. 5)


    Parameters
    ----------
    mu : np.ndarray (dia_mu,) float64
        Unnormalized histogram of dimension `dia_mu`
    b : np.ndarray (dia_nu,) float64
        Unnormalized histograms of dimension `dia_nu`
    M : np.ndarray (dia_mu, dia_nu)
        cost matrix
    reg : float
        Regularization term > 0
    m : float64, optional
        Amount of mass to be transported
    numItermax : int64, optional
        Max number of iterations
    stopThr : float64, optional
        Stop threshold on error (>0)
    verbose : bool, optional
        Print information along iterations
    log : bool, optional
        record log if True


    Returns
    -------
    gamma : (dia_mu, dia_nu) ndarray
        Optimal transportation matrix for the given parameters
    log : dict
        log dictionary returned only if `log` is `True`


    Examples
    --------
    >>> import ot
    >>> mu = [.1, .2]
    >>> nu = [.1, .1]
    >>> M = [[0., 1.], [2., 3.]]
    >>> np.round(entropic_partial_wasserstein(a, b, M, 1, 0.1), 2)
    array([[0.06, 0.02],
           [0.01, 0.  ]])


    .. _references-entropic-partial-wasserstein:
    References
    ----------
    .. [3] Benamou, J. D., Carlier, G., Cuturi, M., Nenna, L., & Peyré, G.
       (2015). Iterative Bregman projections for regularized transportation
       problems. SIAM Journal on Scientific Computing, 37(2), A1111-A1138.

    See Also
    --------
    ot.partial.partial_wasserstein: exact Partial Wasserstein
    """

    mu = np.asarray(mu, dtype=np.float64)
    nu = np.asarray(nu, dtype=np.float64)
    M = np.asarray(M, dtype=np.float64)

    dim_mu, dim_nu = M.shape
     
    if mass < 0:
        raise ValueError("Problem infeasible. Parameter mass should be greater"
                         " than 0.")
    elif mass > np.min(np.stack((np.sum(mu), np.sum(nu)))):
        raise ValueError("Problem infeasible. Parameter m should lower or"
                         " equal than min(|mu|_1, |nu|_1).")
        
            

    # Next 3 lines equivalent to K=np.exp(-M/reg), but faster to compute
    mu_extended  = np.concatenate((mu, np.ones(1)*(np.sum(nu)-mass)))
    nu_extended  = np.concatenate((nu, np.ones(1)*(np.sum(mu)-mass)))
    
    alpha,beta,A=0.0,1.0,np.max(M)+1
    M_extended=np.zeros((n+1,m+1))
    M_extended[0:n,0:m]=M
#    M_extended[0:n,m],M_extended[m,0:m]=alpha,alpha
    M_extended[n,m]=A+2*alpha+beta

    gamma_extended=ot.lp.emd(mu_extended,nu_extended,M_extended,numItermax=numItermax,numThreads=numThreads)
    gamma=gamma_extended[0:n,0:m]
    
    return gamma


In [58]:
#  test the mopt solvers in a toy example: 

n,m=10,15
X=np.random.rand(n,2)*2
Y=np.random.rand(m,2)*2
C=cost_matrix_d(X,Y)
mu=np.random.rand(n)*4
mu/=mu.sum()
nu=np.random.rand(m)*4

nu=nu/nu.sum()*1.2
mass=0.8*min(mu.sum(),nu.sum())

gamma_sinkhorn=sinkhorn_knopp_mopt(mu, nu, C, mass, reg=0.05, numItermax=100000)
gamma_lp=mopt_lp(mu, nu, C, mass, numItermax=100000)



tranas_cost_sinkhorn,penalty_sinkhorn=opt_cost(gamma_sinkhorn,C,Lambda1,Lambda2,mu,nu)
tranas_cost_lp,penalty_lp=opt_cost(gamma_lp,C,Lambda1,Lambda2,mu,nu)

print('mopt_trans_cost from Sinkhorn is %0.2f, transported mass is %0.2f'%(tranas_cost_sinkhorn,gamma_sinkhorn.sum()))
print('mopt_trans_cost from lp is  %0.2f, transported mass is %0.2f '% (tranas_cost_lp,gamma_lp.sum()))




mopt_trans_cost from Sinkhorn is 0.20, transported mass is 0.80
mopt_trans_cost from lp is  0.19, transported mass is 0.80 


In [60]:
# test the specical case OPT problem: 


n,m=10,15
X=np.random.rand(n,2)*2
Y=np.random.rand(m,2)*2
C=cost_matrix_d(X,Y)
mu=np.random.rand(n)*4
mu=mu/mu.sum()*1.2
nu=np.random.rand(m)*4

nu=nu/nu.sum()
mass=min(mu.sum(),nu.sum())
Lambda1=np.array([0.0]) 
Lambda2=np.array([200.0])


gamma_sinkhorn=sinkhorn_knopp_mopt(mu, nu, C, mass, reg=0.05, numItermax=100000)
gamma_lp=mopt_lp(mu, nu, C, mass, numItermax=100000)

tranas_cost_sinkhorn,penalty_sinkhorn=opt_cost(gamma_sinkhorn,C,Lambda1,Lambda2,mu,nu)
tranas_cost_lp,penalty_lp=opt_cost(gamma_lp,C,Lambda1,Lambda2,mu,nu)

print('mopt_trans_cost from Sinkhorn is %0.2f, |gamma2-q| is %0.2f'%(tranas_cost_sinkhorn,np.linalg.norm(nu-gamma_sinkhorn.sum(0))))
print('mopt_trans_cost from lp is  %0.2f, |gamma2-q| is %0.2f '% (tranas_cost_lp,np.linalg.norm(nu-gamma_lp.sum(0))))


gamma_sinkhorn=sinkhorn_knopp_gopt_numba(mu, nu, C, Lambda1,Lambda2, reg=0.01, numItermax=4000,F1='PTV',F2='inf')
tranas_cost_sinkhorn,penalty_sinkhorn=opt_cost(gamma_sinkhorn,C,Lambda1,Lambda2,mu,nu)
gamma_lp=gopt_lp(mu, nu, C, Lambda1,Lambda2, numItermax=2000)
tranas_cost_lp,penalty_lp=opt_cost(gamma_lp,C,Lambda1,Lambda2,mu,nu)

print('gopt_trans_cost from Sinkhorn is %0.2f and |gamma2-q| is %0.2f'%(tranas_cost_sinkhorn,np.linalg.norm(nu-gamma_sinkhorn.sum(0))))
print('gopt_trans_cost from lp  is  %0.2f and |gamma2-q| is %0.2f'% (tranas_cost_lp,np.linalg.norm(nu-gamma_lp.sum(0))))






mopt_trans_cost from Sinkhorn is 0.19, |gamma2-q| is 0.00
mopt_trans_cost from lp is  0.18, |gamma2-q| is 0.00 
gopt_trans_cost from Sinkhorn is 0.18 and 0.00
gopt_trans_cost from lp  is  0.18 and 0.00
